<a href="https://colab.research.google.com/github/weathon/341-final/blob/main/CascadingGPT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install openai

In [ ]:
from openai import OpenAI
from google.colab import userdata
import requests
import pylab

In [ ]:
import json
client = OpenAI(api_key=userdata.get('key'))
final = ""
stack = [{"role":"system","content":"Expect total about 5k words. Create a blog on prompt engineering, ensuring it has a minimum of three layers. Utilize the method of recursion - identifying whether a topic can be addressed in one go or if it necessitates further breakdown. If a topic can be presented directly, proceed with it. However, if it requires further subdivision, create an outline consisting of numerous sub-prompts, calling the main section for these subsections. Make sure that each prompt provides informative content. Deliver responses in JSON format, has to be one of {type:'response', text: 'text here'} or {type:'prompts', text: ['sub prompt1', 'sub prompt2']}. Remember, only JSON outputs are to be provided - avoid placing JSON within code blocks. As only the response (not the prompt) will be visible to the user, ensure to include subtitles in the final response too. Always commence with an outline as your first response."}]
def generate(stack):
  global final
  response = client.chat.completions.create(
    model="gpt-4-1106-preview",
    # model="gpt-3.5-turbo-1106",
    response_format={ "type": "json_object" },
    messages=stack,
  )
  stack.append({"role":"assistant","content":response.choices[0].message.content})
  print(response.choices[0].message.content)
  res = json.loads(response.choices[0].message.content)
  if res["type"]=="response":
    final=(final + res["text"])
    stack = stack[:-1]
    return
  else:
    for i in res["text"]:
      stack.append({"role":"user","content":"Now you write the subsection of "+i})
      generate(stack)
      stack = stack[:-1]
  stack = stack[:-1]

generate(stack)

{
  "type": "prompts",
  "text": [
    "Introduction to Prompt Engineering",
    "The Fundamentals of Prompt Engineering",
    "Advanced Techniques in Prompt Engineering",
    "Practical Applications of Prompt Engineering",
    "The Future of Prompt Engineering"
  ]
}
{
  "type": "prompts",
  "text": [
    "What is Prompt Engineering?",
    "The Importance of Prompt Engineering",
    "The Evolution of Prompt Engineering"
  ]
}


In [ ]:
import json
client = OpenAI(api_key=userdata.get('key'))
final2 = ""
# task = "generate a guidebook for prompt engineering for GPT in Chinese, this should be long and detailed."
def generate(prompt, d):
  global final2
  response = client.chat.completions.create(
  model="gpt-4-1106-preview",
    response_format={ "type": "json_object" },
    messages=[{"role":"system","content":"Create a blog on prompt engineering, ensuring it has a minimum of three layers. Utilize the method of recursion - identifying whether a topic can be addressed in one go or if it necessitates further breakdown. If a topic can be presented directly, proceed with it. However, if it requires further subdivision, create an outline consisting of numerous sub-prompts, calling the main section for these subsections. Make sure that each prompt provides informative content. Deliver responses in JSON format, has to be one of {type:'response', text: 'text here'} or {type:'prompts', text: ['sub prompt1', 'sub prompt2']}. Remember, only JSON outputs are to be provided - avoid placing JSON within code blocks. As only the response (not the prompt) will be visible to the user, ensure to include subtitles in the final response too. Always commence with an outline as your first response."}, {"role":"user","content":"Now write about "+prompt+". And now you are in nested level "+str(d)}],
  )
  res = json.loads(response.choices[0].message.content)
  print(res)
  if res["type"]=="response":
    final2 += (res["text"])
    return
  else:
    for i in res["text"]:
      generate(i, d+1)
generate("The main outline", 0)

In [113]:
response = client.chat.completions.create(
  model="gpt-4-1106-preview",
  messages=[{"role":"user","content":"Judge which book is better. \n\nBook 1:"+final+"\n\nBook2:"+final2}],
  )

In [114]:
print(response.choices[0].message.content)

As an AI developed by OpenAI, I cannot provide subjective opinions or judgments, but I can help you compare and analyze these texts. Both Book 1 and Book 2 offer insights into the field of prompt engineering, but they differ in structure, depth, and the range of topics they cover.

Book 1 provides an extensive, detailed look into the subject, breaking down various aspects of prompt engineering into well-structured chapters. It covers the importance and historical context of prompt engineering, the connection with AI, the roles of prompts in machine learning, effective prompt components, and types. It further delves into techniques and strategies for prompt creation, iterative development, applications across different AI domains, and relevant tools and resources. Notably, it also discusses advanced topics such as chain-of-thought prompting, few-shot and zero-shot learning, and considerations for large language models (LLMs) and transfer learning, along with hyperparameter optimization 

In [115]:
with open("ans.txt","a") as f:
  f.write(f"With path info: {final} Without: {final2} GPT's judge: {response.choices[0].message.content}")